<a id='introduction'></a>
# <p style="padding:15px;background-color:#fff798;margin:10px 0;color:#435672;font-family:'Arial',sans-serif;text-align:center;border-radius:15px 50px;overflow:hidden;font-weight:600">🇪🇺🏛️ European Citizens' Initiative: Commission Response</p>

<div align="center">
  <img src="LOGO CE_RGB_MUTE_POS.svg" alt="EU Commission Logo" height="200" style="display:inline-block; margin:10px;">
  <img src="1_2021_1-1.jpg" alt="ECI Material" height="200" style="display:inline-block; margin:10px;">
</div>

<p style="text-align:center;">
  <i>Source: European Citizens' Initiative | European Commission (CC BY 4.0)</i>
</p>

Examines Commission responses to [European Citizens' Initiative proposals](https://commission.europa.eu/get-involved/engage-eu-policymaking/european-citizens-initiative_en) that successfully met signature thresholds between 2012 and 2025. Once an ECI collects 1 million signatures from at least 7 member states, the Commission must provide a formal response within 6 months explaining whether it will propose new legislation. This dataset tracks 11 out of 16 ECIs that successfully met both signature criteria (1M+ signatures and 7-country thresholds)—analyzing Commission response types, implementation timelines, parliamentary engagement, and follow-up actions.

This analysis focuses exclusively on what happens after ECIs meet signature requirements, building upon the previous [**🇪🇺✍️ European Citizens' Initiative: Signature Collection**]() study which examined all 121 registered ECIs. It does not cover the registration approval process itself, including which proposed ECIs were refused registration or how to prepare a successful registration application—[more about this](https://citizens-initiative.europa.eu/how-it-works_en).

Success in this analysis is measured by Commission outcome categories (Law Active, Law Promised, Rejected, etc.) and implementation status, not by whether proposals were substantively "correct" or how individual organizers interpret their outcomes.

NOTE:<br>
> If you're interested in understanding how laws are passed at the EU level and how much power each institution holds, watch this [10-minutes explanation](https://www.youtube.com/watch?v=cotxhOkux18). It would be helpful to understand the feasibility of ECI.

<a id='table-of-contents'></a>
## <p style="padding:10px;background-color:#fff798;margin:0;color:#435672;font-family:newtimesroman;text-align:center;border-radius: 15px 50px;overflow:hidden;font-weight:500">🧭 Table of content</p>

[🌟 **Introduction**](#introduction)

[❓ **Questions to Ask:**](#question-1)
- [1. Success Patterns](#question-1)
- [2. Temporal Patterns](#question-2)
- [3. Parliament Actions](#question-3)
- [4. Funding Patterns](#question-4)
- [5. Geographic Strategies](#question-5)
- [6. Organizational](#question-6)
- [7. Content Features](#question-7)
- [8. Commission Engagement](#question-8)
- [9. Response Mechanisms](#question-9)
- [10. Key Findings](#question-10)


<a id='setup'></a>
## <p style="padding:10px;background-color:#fff798;margin:0;color:#435672;font-family:newtimesroman;text-align:center;border-radius: 15px 50px;overflow:hidden;font-weight:500">⚙️ Setup: Import Libraries and Load Data</p>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import json
import warnings
warnings.filterwarnings('ignore')

# Set defaults
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("husl")
pd.set_option('display.max_columns', None)

# Load main datasets
# Load the dataset
data_folder = "../data/2025-09-18_16-33-57"

df_initiatives = pd.read_csv(f'{data_folder}/eci_initiatives_2025-11-04_11-59-38.csv')
df_merger = pd.read_csv(f'{data_folder}/eci_merger_responses_and_followup_2025-12-15_15-33-12.csv')

print(f"✓ Initiatives file: {df_initiatives.shape[0]} ECIs")
print(f"✓ Merger file: {df_merger.shape[0]} Commission responses")
print(f"\nColumns: {len(df_initiatives.columns)} initiative columns")
print(f"        {len(df_merger.columns)} response columns")

✓ Initiatives file: 121 ECIs
✓ Merger file: 11 Commission responses

Columns: 26 initiative columns
        36 response columns


<a id='data-cleaning'></a>
## <p style="padding:10px;background-color:#fff798;margin:0;color:#435672;font-family:newtimesroman;text-align:center;border-radius: 15px 50px;overflow:hidden;font-weight:500">🧹 Data Cleaning and Feature Engineering</p>

In [2]:
# ==============================================================================
# DATA CLEANING AND FEATURE ENGINEERING
# ==============================================================================

import pandas as pd
import numpy as np
import json

# ------------------------------------------------------------------------------
# Signature Data Parsing and Conversion
# ------------------------------------------------------------------------------

# Parse signatures (handle commas and convert to numeric)
def parse_signatures(sig):
    """Convert signature strings with commas to numeric values."""
    if pd.isna(sig):
        return np.nan
    if isinstance(sig, str):
        return float(sig.replace(',', ''))
    return float(sig)

# Apply to both dataframes
df_initiatives['signatures_numeric'] = df_initiatives['signatures_collected'].apply(parse_signatures)
df_initiatives['signatures_threshold_met_numeric'] = pd.to_numeric(
    df_initiatives['signatures_threshold_met'], errors='coerce'
)

# Same for merger if it has signatures
if 'signatures_collected' in df_merger.columns:
    df_merger['signatures_numeric'] = df_merger['signatures_collected'].apply(parse_signatures)

# ------------------------------------------------------------------------------
# Funding Data Parsing and Conversion
# ------------------------------------------------------------------------------

# Parse funding (handle commas and convert to numeric)
def parse_funding(funding):
    """Convert funding strings with commas to numeric values."""
    if pd.isna(funding):
        return 0
    if isinstance(funding, str):
        return float(funding.replace(',', ''))
    return float(funding)

if 'funding_total' in df_initiatives.columns:
    df_initiatives['funding_numeric'] = df_initiatives['funding_total'].apply(parse_funding)

# ------------------------------------------------------------------------------
# Date Parsing and Conversion
# ------------------------------------------------------------------------------

# Parse date strings from DD/MM/YYYY to datetime
def parse_date(date_str):
    """Convert date string to datetime object."""
    if pd.isna(date_str) or date_str == '':
        return pd.NaT
    try:
        return pd.to_datetime(date_str, format='%d/%m/%Y')
    except:
        # Try default parser as fallback
        try:
            return pd.to_datetime(date_str, errors='coerce')
        except:
            return pd.NaT

# Parse main timeline dates for df_initiatives
date_cols = ['timeline_registered', 'timeline_collection_start_date', 
             'timeline_collection_closed', 'timeline_verification_start',
             'timeline_verification_end', 'timeline_response_commission_date']

for col in date_cols:
    if col in df_initiatives.columns:
        df_initiatives[col] = df_initiatives[col].apply(parse_date)

# Extract year from registration
df_initiatives['registration_year'] = df_initiatives['timeline_registered'].dt.year

# ------------------------------------------------------------------------------
# Duration Calculations
# ------------------------------------------------------------------------------

# Calculate durations
df_initiatives['collection_days'] = (
    df_initiatives['timeline_collection_closed'] - 
    df_initiatives['timeline_collection_start_date']
).dt.days

df_initiatives['verification_days'] = (
    df_initiatives['timeline_verification_end'] - 
    df_initiatives['timeline_verification_start']
).dt.days

df_initiatives['time_to_response_days'] = (
    df_initiatives['timeline_response_commission_date'] - 
    df_initiatives['timeline_registered']
).dt.days

# ------------------------------------------------------------------------------
# Success Metrics Definition
# ------------------------------------------------------------------------------

# Define success categories
df_initiatives['reached_signatures'] = df_initiatives['signatures_numeric'] >= 1000000
df_initiatives['met_country_threshold'] = df_initiatives['signatures_threshold_met_numeric'] >= 7
df_initiatives['successful_eci'] = (
    df_initiatives['reached_signatures'] & 
    df_initiatives['met_country_threshold']
)

# Define success outcomes
success_outcomes = ['Commission Response', 'Answered initiative', 'Valid initiative']
df_initiatives['is_successful'] = df_initiatives['final_outcome'].isin(success_outcomes).astype(int)

# ------------------------------------------------------------------------------
# Create Signature Volume Categories
# ------------------------------------------------------------------------------

df_initiatives['signature_category'] = pd.cut(
    df_initiatives['signatures_numeric'],
    bins=[0, 100000, 500000, 1000000, float('inf')],
    labels=['<100k', '100k-500k', '500k-1M', '>1M'],
    include_lowest=True
)

# ------------------------------------------------------------------------------
# JSON Fields Parsing
# ------------------------------------------------------------------------------

def safe_json_load(x):
    """Safely parse JSON strings."""
    try:
        return json.loads(x) if pd.notna(x) else None
    except:
        return None

# Parse organizer data
if 'organizer_representative' in df_initiatives.columns:
    df_initiatives['organizer_data'] = df_initiatives['organizer_representative'].apply(safe_json_load)

print("✅ Data cleaning complete!")
print(f"   - Signatures converted: {df_initiatives['signatures_numeric'].notna().sum()} records")
print(f"   - Date columns parsed: {len(date_cols)} columns")
print(f"   - Successful ECIs: {df_initiatives['successful_eci'].sum()} out of {len(df_initiatives)}")


✅ Data cleaning complete!
   - Signatures converted: 60 records
   - Date columns parsed: 6 columns
   - Successful ECIs: 16 out of 121


<a id='question-1'></a>
## <p style="padding:10px;background-color:#fff798;margin:0;color:#435672;font-family:newtimesroman;text-align:center;border-radius: 15px 50px;overflow:hidden;font-weight:500">1. What are the success patterns and outcome distributions for ECIs?</p>

<div class="analysis-components">
    <strong>Visualizations:</strong>
    <ul>
        <li>Pie chart: Outcome distribution (Law Active, Rejected, etc.)</li>
        <li>Scatter plot: Total signatures vs outcome type</li>
        <li>Grouped bar: Success rate by signature volume</li>
        <li>Data tables: Outcome counts, signature statistics</li>
    </ul>
</div>

In [3]:
# Pie chart: Outcome distribution (Law Active, Rejected, etc.) + Still Waiting
import plotly.graph_objects as go
import pandas as pd

# Find ECIs that met thresholds but haven't received Commission response yet
successful_ecis = df_initiatives[df_initiatives['successful_eci'] == True]['registration_number'].tolist()
responded_ecis = df_merger['registration_number'].tolist()
waiting_ecis = [eci for eci in successful_ecis if eci not in responded_ecis]
waiting_count = len(waiting_ecis)

# Merge to get ECI titles for hover
merged_data = df_merger.merge(
    df_initiatives[['registration_number', 'title']], 
    on='registration_number', 
    how='left'
)

# Prepare data for responded ECIs
outcome_counts = df_merger['final_outcome_status'].value_counts().reset_index()
outcome_counts.columns = ['Outcome', 'Count']

# Add "Waiting for Response" row
if waiting_count > 0:
    waiting_row = pd.DataFrame({
        'Outcome': ['Waiting for Response'],
        'Count': [waiting_count]
    })
    outcome_counts = pd.concat([outcome_counts, waiting_row], ignore_index=True)

# Calculate total (responded + waiting)
total_count = len(df_merger) + waiting_count
outcome_counts['Percentage'] = (outcome_counts['Count'] / total_count * 100).round(1)

# Define outcome ranking for color assignment (higher = better)
outcome_ranking = {
    'Law Active': 5,                            # Best - Dark green
    'Law Approved': 4,                          # Very good - Green
    'Law Promised': 3,                          # Good - Light green
    'Action Plan Created': 2,                   # Neutral - Yellow
    'Waiting for Response': 0,                  # Unknown/waiting - Gray
    'Being Studied': 1,                         # Neutral-negative - Orange
    'Rejected - Alternative Actions': -1,       # Bad - Light red
    'Rejected - Already Covered': -2,           # Worst - Dark red
}

# Add ranking to dataframe
outcome_counts['Rank'] = outcome_counts['Outcome'].map(outcome_ranking)

# Create gradient colors based on ranking
def get_outcome_color(outcome):
    """Generate color based on outcome (green=good, yellow=neutral, red=bad)."""
    if outcome == 'Law Active':
        return 'rgb(60, 163, 113)'  # Dark green
    elif outcome == 'Law Approved':
        return 'rgb(102, 187, 106)'  # Medium green
    elif outcome == 'Law Promised':
        return 'rgb(156, 204, 101)'  # Light green
    elif outcome == 'Action Plan Created':
        return 'rgb(255, 193, 7)'  # Yellow
    elif outcome == 'Being Studied':
        return 'rgb(255, 152, 0)'  # Orange
    elif outcome == 'Rejected - Alternative Actions':
        return 'rgb(244, 67, 54)'  # Light red
    elif outcome == 'Rejected - Already Covered':
        return 'rgb(183, 28, 28)'  # DARK RED
    elif outcome == 'Waiting for Response':
        return 'rgb(158, 158, 158)'  # Gray (unknown status)
    else:
        return 'rgb(117, 117, 117)'  # Darker gray for unknown

outcome_counts['Color'] = outcome_counts['Outcome'].apply(get_outcome_color)

# Prepare ECI lists for hover
def prepare_eci_list_for_hover(ecis, max_items=15):
    """Prepare ECI title list for hover tooltips with truncation."""
    if not ecis:
        return "No ECIs"
    elif len(ecis) <= max_items:
        return '<br>'.join(f"• {title}" for title in ecis)
    else:
        text = '<br>'.join(f"• {title}" for title in ecis[:max_items])
        text += f"<br><i>... (and {len(ecis) - max_items} more)</i>"
        return text

eci_lists = []
for outcome in outcome_counts['Outcome']:
    if outcome == 'Waiting for Response':
        # Get titles of waiting ECIs
        waiting_titles = df_initiatives[
            df_initiatives['registration_number'].isin(waiting_ecis)
        ]['title'].tolist()
        eci_lists.append(prepare_eci_list_for_hover(waiting_titles))
    else:
        ecis = merged_data[merged_data['final_outcome_status'] == outcome]['title'].tolist()
        eci_lists.append(prepare_eci_list_for_hover(ecis))

outcome_counts['ECI_List'] = eci_lists

# Sort by rank (best outcomes first)
outcome_counts = outcome_counts.sort_values('Rank', ascending=False)

# Create pie chart
fig = go.Figure(go.Pie(
    labels=outcome_counts['Outcome'],
    values=outcome_counts['Count'],
    hole=0.1,
    marker=dict(colors=outcome_counts['Color'].tolist()),
    customdata=outcome_counts['ECI_List'],
    hovertemplate='<b>%{label}</b><br>' +
                  'Count: %{value}<br>' +
                  'Percentage: %{percent}<br><br>' +
                  '<b>ECIs:</b><br>%{customdata}' +
                  '<extra></extra>',
    textinfo='percent+label',
    textposition='inside',
    textfont=dict(size=11, color='white', family='Arial Black'),
    sort=False  # Keep our custom sort order
))

fig.update_layout(
    title=f'<b>Commission Response Outcomes ({len(df_merger)} responded, {waiting_count} waiting)</b>',
    height=600,
    showlegend=True,
    legend=dict(
        font=dict(size=12),
        orientation='v',
        yanchor='middle',
        y=0.5,
        xanchor='left',
        x=1.02
    )
)

fig.show()

In [4]:
# Scatter plot: Total signatures vs outcome type (ENHANCED WITH TIMELINE)
import plotly.express as px
import pandas as pd
import numpy as np

# STEP 1: Merge with all needed timeline columns
merged_data = df_merger.merge(
    df_initiatives[['registration_number', 'signatures_numeric', 'title', 
                    'timeline_registered', 'timeline_collection_closed']], 
    on='registration_number', 
    how='left'
)

# Get the commission response date
if 'official_communication_adoption_date' in df_merger.columns:
    merged_data['response_date'] = pd.to_datetime(df_merger['official_communication_adoption_date'], errors='coerce')
elif 'commission_submission_date' in df_merger.columns:
    merged_data['response_date'] = pd.to_datetime(df_merger['commission_submission_date'], errors='coerce')

# Convert timeline columns to datetime
merged_data['registration_date'] = pd.to_datetime(merged_data['timeline_registered'], errors='coerce')
merged_data['collection_closed_date'] = pd.to_datetime(merged_data['timeline_collection_closed'], errors='coerce')

# Calculate wait time between collection closed and response
def format_duration(start_date, end_date):
    """Format duration as 'X years Y months Z days' or 'X days'."""
    if pd.isna(start_date) or pd.isna(end_date):
        return 'Unknown'
    
    delta = (end_date - start_date).days
    
    if delta < 0:
        return 'Invalid date range'
    
    years = delta // 365
    remaining = delta % 365
    months = remaining // 30
    days = remaining % 30
    
    parts = []
    if years > 0:
        parts.append(f"{years} year{'s' if years > 1 else ''}")
    if months > 0:
        parts.append(f"{months} month{'s' if months > 1 else ''}")
    if days > 0 or len(parts) == 0:
        parts.append(f"{days} day{'s' if days != 1 else ''}")
    
    return ' '.join(parts)

merged_data['wait_time'] = merged_data.apply(
    lambda row: format_duration(row['collection_closed_date'], row['response_date']), 
    axis=1
)

# Format dates for display
merged_data['registration_formatted'] = merged_data['registration_date'].dt.strftime('%d %b %Y')
merged_data['collection_closed_formatted'] = merged_data['collection_closed_date'].dt.strftime('%d %b %Y')
merged_data['response_date_formatted'] = merged_data['response_date'].dt.strftime('%d %b %Y')

# Fill NaN with readable text
merged_data['registration_formatted'] = merged_data['registration_formatted'].fillna('Unknown')
merged_data['collection_closed_formatted'] = merged_data['collection_closed_formatted'].fillna('Unknown')
merged_data['response_date_formatted'] = merged_data['response_date_formatted'].fillna('Unknown')

# Define outcome ranking and colors (same as pie chart)
outcome_ranking = {
    'Law Active': 5,
    'Law Approved': 4,
    'Law Promised': 3,
    'Action Plan Created': 2,
    'Being Studied': 1,
    'Rejected - Alternative Actions': -1,
    'Rejected - Already Covered': -2,
}

outcome_colors = {
    'Law Active': 'rgb(60, 163, 113)',
    'Law Approved': 'rgb(102, 187, 106)',
    'Law Promised': 'rgb(156, 204, 101)',
    'Action Plan Created': 'rgb(255, 193, 7)',
    'Being Studied': 'rgb(255, 152, 0)',
    'Rejected - Alternative Actions': 'rgb(244, 67, 54)',
    'Rejected - Already Covered': 'rgb(183, 28, 28)',
}

# Add ranking for sorting
merged_data['outcome_rank'] = merged_data['final_outcome_status'].map(outcome_ranking)

# Apply power scaling for bubble sizes
merged_data['size_scaled'] = merged_data['signatures_numeric'] ** 4.3

# Sort outcomes by rank (best to worst) for legend ordering
outcome_order = sorted(
    merged_data['final_outcome_status'].unique(), 
    key=lambda x: outcome_ranking.get(x, 0), 
    reverse=True
)

# Create categorical x-axis positions
merged_data['outcome_position'] = merged_data['final_outcome_status'].map(
    {outcome: i for i, outcome in enumerate(outcome_order)}
)

# Create scatter plot
fig = px.scatter(
    merged_data.sort_values('outcome_rank', ascending=False),
    x='outcome_position',
    y='signatures_numeric',
    size='size_scaled',
    color='final_outcome_status',
    custom_data=['title', 'registration_formatted', 'collection_closed_formatted', 
                 'response_date_formatted', 'wait_time', 'final_outcome_status', 
                 'signatures_numeric'],
    title=f'<b>Signatures vs Commission Response Outcome (Responses: {len(merged_data)})</b>',
    labels={
        'outcome_position': 'Outcome Category',
        'signatures_numeric': 'Total Signatures Collected',
        'final_outcome_status': 'Outcome'
    },
    category_orders={'final_outcome_status': outcome_order},
    color_discrete_map=outcome_colors,
    size_max=60
)

# Custom hover template with full timeline
fig.update_traces(
    hovertemplate=(
        '<b>%{customdata[0]}:</b><br><br>'
        '<b>Outcome:</b> %{customdata[5]}<br>'
        '<b>Signatures:</b> %{customdata[6]:,.0f}<br><br>'
        '<b>Registration:</b> %{customdata[1]}<br>'
        '<b>Signatures Ended:</b> %{customdata[2]}<br>'
        '<b>Response Date:</b> %{customdata[3]}<br>'
        '<b>Response Wait Time:</b> %{customdata[4]}<br>'
        '<extra></extra>'
    )
)

# Update x-axis to show outcome labels
fig.update_xaxes(
    tickmode='array',
    tickvals=list(range(len(outcome_order))),
    ticktext=outcome_order,
    tickangle=45,
    title=dict(text='Outcome Category', font=dict(size=14))
)

# Format y-axis with comma separators
fig.update_yaxes(
    title=dict(text='Total Signatures Collected', font=dict(size=14)),
    tickformat=','
)

fig.update_layout(
    height=650,
    showlegend=True,
    legend=dict(
        title=dict(text='<b>Outcome</b>', font=dict(size=12)),
        font=dict(size=11),
        orientation='v',
        yanchor='top',
        y=1,
        xanchor='left',
        x=1.02
    ),
    hovermode='closest'
)

fig.show()

In [18]:
# Scatter plot: Total signatures vs outcome type (ENHANCED WITH TIMELINE + FORMATTED OBJECTIVES)
import plotly.express as px
import pandas as pd
import numpy as np

# STEP 1: Merge with all needed timeline columns + objective
merged_data = df_merger.merge(
    df_initiatives[['registration_number', 'signatures_numeric', 'title', 
                    'timeline_registered', 'timeline_collection_closed', 'objective']], 
    on='registration_number', 
    how='left'
)

# Get the commission response date
if 'official_communication_adoption_date' in df_merger.columns:
    merged_data['response_date'] = pd.to_datetime(df_merger['official_communication_adoption_date'], errors='coerce')
elif 'commission_submission_date' in df_merger.columns:
    merged_data['response_date'] = pd.to_datetime(df_merger['commission_submission_date'], errors='coerce')

# Convert timeline columns to datetime
merged_data['registration_date'] = pd.to_datetime(merged_data['timeline_registered'], errors='coerce')
merged_data['collection_closed_date'] = pd.to_datetime(merged_data['timeline_collection_closed'], errors='coerce')

# Calculate wait time between collection closed and response
def format_duration(start_date, end_date):
    """Format duration as 'X years Y months Z days' or 'X days'."""
    if pd.isna(start_date) or pd.isna(end_date):
        return 'Unknown'
    
    delta = (end_date - start_date).days
    
    if delta < 0:
        return 'Invalid date range'
    
    years = delta // 365
    remaining = delta % 365
    months = remaining // 30
    days = remaining % 30
    
    parts = []
    if years > 0:
        parts.append(f"{years} year{'s' if years > 1 else ''}")
    if months > 0:
        parts.append(f"{months} month{'s' if months > 1 else ''}")
    if days > 0 or len(parts) == 0:
        parts.append(f"{days} day{'s' if days != 1 else ''}")
    
    return ' '.join(parts)

# Format objective with line breaks every 11 words
def format_objective(objective):
    if pd.isna(objective):
        return "No objective provided"
    
    obj_str = str(objective)
    
    # Split by bullet points first
    bullet_sections = obj_str.split('•')
    formatted_sections = []
    
    for i, section in enumerate(bullet_sections):
        section = section.strip()
        if not section:
            continue
            
        # Add bullet back (except for first section if it didn't start with bullet)
        if i > 0:
            section = '• ' + section
        
        # Add line breaks every 11 words within each section
        words = section.split()
        lines = []
        for j in range(0, len(words), 11):
            lines.append(' '.join(words[j:j+11]))
        
        formatted_sections.append('<br>'.join(lines))
    
    return '<br>'.join(formatted_sections)

merged_data['wait_time'] = merged_data.apply(
    lambda row: format_duration(row['collection_closed_date'], row['response_date']), 
    axis=1
)

# Format dates for display
merged_data['registration_formatted'] = merged_data['registration_date'].dt.strftime('%d %b %Y')
merged_data['collection_closed_formatted'] = merged_data['collection_closed_date'].dt.strftime('%d %b %Y')
merged_data['response_date_formatted'] = merged_data['response_date'].dt.strftime('%d %b %Y')

# Fill NaN with readable text
merged_data['registration_formatted'] = merged_data['registration_formatted'].fillna('Unknown')
merged_data['collection_closed_formatted'] = merged_data['collection_closed_formatted'].fillna('Unknown')
merged_data['response_date_formatted'] = merged_data['response_date_formatted'].fillna('Unknown')

# Apply objective formatting
merged_data['objective_display'] = merged_data['objective'].apply(format_objective)

# Define outcome ranking and colors (same as pie chart)
outcome_ranking = {
    'Law Active': 5,
    'Law Approved': 4,
    'Law Promised': 3,
    'Action Plan Created': 2,
    'Being Studied': 1,
    'Rejected - Alternative Actions': -1,
    'Rejected - Already Covered': -2,
}

outcome_colors = {
    'Law Active': 'rgb(60, 163, 113)',
    'Law Approved': 'rgb(102, 187, 106)',
    'Law Promised': 'rgb(156, 204, 101)',
    'Action Plan Created': 'rgb(255, 193, 7)',
    'Being Studied': 'rgb(255, 152, 0)',
    'Rejected - Alternative Actions': 'rgb(244, 67, 54)',
    'Rejected - Already Covered': 'rgb(183, 28, 28)',
}

# Add ranking for sorting
merged_data['outcome_rank'] = merged_data['final_outcome_status'].map(outcome_ranking)

# Apply power scaling for bubble sizes
merged_data['size_scaled'] = merged_data['signatures_numeric'] ** 4.3

# Sort outcomes by rank (best to worst) for legend ordering
outcome_order = sorted(
    merged_data['final_outcome_status'].unique(), 
    key=lambda x: outcome_ranking.get(x, 0), 
    reverse=True
)

# Create categorical x-axis positions
merged_data['outcome_position'] = merged_data['final_outcome_status'].map(
    {outcome: i for i, outcome in enumerate(outcome_order)}
)

# Create scatter plot
fig = px.scatter(
    merged_data.sort_values('outcome_rank', ascending=False),
    x='outcome_position',
    y='signatures_numeric',
    size='size_scaled',
    color='final_outcome_status',
    custom_data=['title', 'registration_formatted', 'collection_closed_formatted', 
                 'response_date_formatted', 'wait_time', 'final_outcome_status', 
                 'signatures_numeric', 'objective_display'],
    title=f'<b>Signatures vs Commission Response Outcome (Responses: {len(merged_data)})</b>',
    labels={
        'outcome_position': 'Outcome Category',
        'signatures_numeric': 'Total Signatures Collected',
        'final_outcome_status': 'Outcome'
    },
    category_orders={'final_outcome_status': outcome_order},
    color_discrete_map=outcome_colors,
    size_max=60
)

# Custom hover template with full timeline + formatted objectives
fig.update_traces(
    hovertemplate=(
        '<b>%{customdata[0]}</b><br><br>'
        '<b>Outcome:</b> %{customdata[5]}<br>'
        '<b>Signatures:</b> %{customdata[6]:,.0f}<br><br>'
        '<b>Registration:</b> %{customdata[1]}<br>'
        '<b>Signatures Ended:</b> %{customdata[2]}<br>'
        '<b>Response Date:</b> %{customdata[3]}<br>'
        '<b>Response Wait Time:</b> %{customdata[4]}<br><br>'
        '<b>Objective:</b><br>%{customdata[7]}<br>'
        '<extra></extra>'
    )
)

# Update x-axis to show outcome labels
fig.update_xaxes(
    tickmode='array',
    tickvals=list(range(len(outcome_order))),
    ticktext=outcome_order,
    tickangle=45,
    title=dict(text='Outcome Category', font=dict(size=14))
)

# Format y-axis with comma separators
fig.update_yaxes(
    title=dict(text='Total Signatures Collected', font=dict(size=14)),
    tickformat=','
)

fig.update_layout(
    height=650,
    showlegend=True,
    legend=dict(
        title=dict(text='<b>Outcome</b>', font=dict(size=12)),
        font=dict(size=11),
        orientation='v',
        yanchor='top',
        y=1,
        xanchor='left',
        x=1.02
    ),
    hovermode='closest'
)

fig.show()


In [6]:
# Dataframe: All Rejected ECIs with Full Reasons
import pandas as pd

# Define rejection outcome types
REJECTION_OUTCOMES = ['Rejected - Already Covered', 'Rejected - Alternative Actions']

# Filter for rejected ECIs
rejected_ecis = df_merger[df_merger['final_outcome_status'].isin(REJECTION_OUTCOMES)].copy()

# Merge with initiatives to get full title and signatures
rejected_ecis = rejected_ecis.merge(
    df_initiatives[['registration_number', 'title', 'signatures_numeric']], 
    on='registration_number', 
    how='left'
)

# Create clean summary dataframe
rejected_summary = rejected_ecis[[
    'registration_number',
    'title',
    'signatures_numeric',
    'final_outcome_status',
    'commission_rejection_reason'  # Using the correct column
]].copy()

# Rename columns for display
rejected_summary.columns = [
    'Registration Number',
    'ECI Title',
    'Signatures',
    'Rejection Type',
    'Rejection Reason'
]

# Format signatures with commas
rejected_summary['Signatures Formatted'] = rejected_summary['Signatures'].apply(
    lambda x: f'{x:,.0f}' if pd.notna(x) else 'N/A'
)

# Sort by signatures (descending) to show the paradox
rejected_summary = rejected_summary.sort_values('Signatures', ascending=False)

# Configure pandas to show full text
pd.set_option('display.max_colwidth', None)

display(rejected_summary[[
    'Registration Number', 
    'ECI Title', 
    'Signatures Formatted',
    'Rejection Type', 
    'Rejection Reason'
]].style.set_properties(**{
    'text-align': 'left',
    'white-space': 'pre-wrap'
}).hide(axis='index'))

# Reset pandas display option
pd.reset_option('display.max_colwidth')


Registration Number,ECI Title,Signatures Formatted,Rejection Type,Rejection Reason
2012/000005,One of us,"1,721,626",Rejected - Already Covered,The Commission decided not to make a legislative proposal.
2019/000007,Cohesion policy for the equality of the regions and sustainability of the regional cultures,"1,269,351",Rejected - Already Covered,"The Commission carefully analysed the citizens' proposals and concluded that while some proposals fall outside of EU competence, as they would interfere with the existing constitutional setup of the concerned Member States, others are already covered under the current Cohesion policy thanks to its robust safeguards promoting inclusion and equal treatment of minorities, as well as the respect for cultural and linguistic diversity."
2012/000007,Stop vivisection,"1,173,130",Rejected - Alternative Actions,"While the Commission does share the conviction that animal testing should be phased out in Europe, its approach for achieving that objective differs from the one proposed in this Citizens' Initiative. The Commission considers that the Directive on the protection of animals used for scientific purposes (Directive 2010/63/EU), which the Initiative seeks to repeal, is the right legislation to achieve the underlying objectives of the Initiative. It sets full replacement of animals as its ultimate goal as soon as it is scientifically possibly, and provides a legally binding stepwise approach as non-animal alternatives become available. Therefore, no repeal of that legislation was proposed."
2017/000004,Minority SafePack – one million signatures for diversity in Europe,"1,123,422",Rejected - Already Covered,The Commission decided not to make a legislative proposal.


In [7]:
# Create interpretable comparison dataframe with ECI titles and VALIDATION
import pandas as pd

# Merge to get titles
merged_data = df_merger.merge(
    df_initiatives[['registration_number', 'signatures_numeric', 'title']], 
    on='registration_number', 
    how='left'
)

# Find the specific ECIs
highest_sig_eci = merged_data.loc[merged_data['signatures_numeric'].idxmax()]
lowest_sig_eci = merged_data.loc[merged_data['signatures_numeric'].idxmin()]

# Define success and rejection categories for validation
SUCCESS_OUTCOMES = ['Law Active', 'Law Approved', 'Law Promised']
REJECTION_OUTCOMES = ['Rejected - Already Covered', 'Rejected - Alternative Actions']
NEUTRAL_OUTCOMES = ['Being Studied', 'Action Plan Created']

# VALIDATION: Check if our assumption holds
highest_outcome = highest_sig_eci['final_outcome_status']
lowest_outcome = lowest_sig_eci['final_outcome_status']

# Validate highest signature ECI (should be rejected or neutral, NOT successful)
if highest_outcome in SUCCESS_OUTCOMES:
    raise ValueError(
        f"❌ VALIDATION ERROR: Highest signature ECI ({highest_sig_eci['signatures_numeric']:,.0f}) "
        f"has SUCCESS outcome '{highest_outcome}'.\n"
        f"This contradicts the 'signature paradox' narrative. Review your interpretation."
    )

# Validate lowest signature ECI (should be successful or neutral, NOT rejected)
if lowest_outcome in REJECTION_OUTCOMES:
    raise ValueError(
        f"❌ VALIDATION ERROR: Lowest signature ECI ({lowest_sig_eci['signatures_numeric']:,.0f}) "
        f"has REJECTION outcome '{lowest_outcome}'.\n"
        f"This contradicts the 'signature paradox' narrative. Review your interpretation."
    )

# Determine appropriate emojis based on outcome
def get_outcome_emoji(outcome):
    """Return appropriate emoji based on outcome category."""
    if outcome in SUCCESS_OUTCOMES:
        return "✅"
    elif outcome in REJECTION_OUTCOMES:
        return "❌"
    elif outcome in NEUTRAL_OUTCOMES:
        return "⏳"
    else:
        raise ValueError(
            f"❌ UNKNOWN OUTCOME ERROR: Encountered unexpected outcome '{outcome}'.\n"
            f"Known outcomes are:\n"
            f"  - Success: {SUCCESS_OUTCOMES}\n"
            f"  - Rejection: {REJECTION_OUTCOMES}\n"
            f"  - Neutral: {NEUTRAL_OUTCOMES}\n"
            f"Please add this outcome to the appropriate category or investigate if it's a data quality issue."
        )

# Key findings that demonstrate signature count paradox
signature_paradox_data = {
    'Finding': [
        '🏆 Highest signature count',
        '📉 Lowest signature count',
    ],
    
    'ECI Title': [
        highest_sig_eci['title'],
        lowest_sig_eci['title'],
    ],
    
    'Signatures': [
        f"{highest_sig_eci['signatures_numeric']:,.0f}",
        f"{lowest_sig_eci['signatures_numeric']:,.0f}",
    ],
    
    'Outcome': [
        f"{get_outcome_emoji(highest_outcome)} {highest_outcome}",
        f"{get_outcome_emoji(lowest_outcome)} {lowest_outcome}",
    ],
    
    'Explanation': [
        'Most signatures → ' + ('Still rejected' if highest_outcome in REJECTION_OUTCOMES else 'Outcome: ' + highest_outcome),
        'Least signatures → ' + ('Full success' if lowest_outcome == 'Law Active' else 'Outcome: ' + lowest_outcome),
    ]
}

df_paradox = pd.DataFrame(signature_paradox_data)

# Display
print("=" * 120)
print("📊 SIGNATURE PARADOX: The Two Most Extreme Cases")
print("=" * 120)
display(df_paradox.style.set_properties(**{
    'text-align': 'left',
    'white-space': 'pre-wrap'
}).hide(axis='index'))

📊 SIGNATURE PARADOX: The Two Most Extreme Cases


Finding,ECI Title,Signatures,Outcome,Explanation
🏆 Highest signature count,One of us,"1,721,626",❌ Rejected - Already Covered,Most signatures → Still rejected
📉 Lowest signature count,Save bees and farmers ! Towards a bee-friendly agriculture for a healthy environment,"1,054,973",✅ Law Active,Least signatures → Full success


In [8]:
# Additional summary statistics - DYNAMICALLY CALCULATED
print("\n📈 STATISTICAL SUMMARY")
print("-"*100)

# Calculate stats for Law Active
law_active_stats = merged_data[merged_data['final_outcome_status'] == 'Law Active']['signatures_numeric']
law_active_count = len(law_active_stats)
law_active_min = law_active_stats.min()
law_active_mean = law_active_stats.mean()
law_active_max = law_active_stats.max()

# Calculate stats for Rejected - Already Covered
rejected_stats = merged_data[merged_data['final_outcome_status'] == 'Rejected - Already Covered']['signatures_numeric']
rejected_count = len(rejected_stats)
rejected_min = rejected_stats.min()
rejected_mean = rejected_stats.mean()
rejected_max = rejected_stats.max()

# Calculate differences
diff_min = rejected_min - law_active_min
diff_mean = rejected_mean - law_active_mean
diff_max = rejected_max - law_active_max

# Determine insights
count_comparison = 'Same' if law_active_count == rejected_count else f'{rejected_count} vs {law_active_count}'
min_insight = f'+{diff_min:,.0f} more for rejected' if diff_min > 0 else f'{abs(diff_min):,.0f} less for rejected'
mean_insight = f'+{diff_mean:,.0f} more for rejected' if diff_mean > 0 else f'{abs(diff_mean):,.0f} less for rejected'
max_insight = f'+{diff_max:,.0f} more for rejected' if diff_max > 0 else f'{abs(diff_max):,.0f} less for rejected'

summary_comparison = pd.DataFrame({
    'Outcome Category': [
        'Law Active (Best)', 
        'Rejected - Already Covered (Worst)', 
        'Difference'
    ],
    'Count': [
        law_active_count,
        rejected_count,
        count_comparison
    ],
    'Min Signatures': [
        f'{law_active_min:,.0f}',
        f'{rejected_min:,.0f}',
        min_insight
    ],
    'Mean Signatures': [
        f'{law_active_mean:,.0f}',
        f'{rejected_mean:,.0f}',
        mean_insight
    ],
    'Max Signatures': [
        f'{law_active_max:,.0f}',
        f'{rejected_max:,.0f}',
        max_insight
    ],
    'Key Insight': [
        'Success with LEAST signatures' if law_active_mean < rejected_mean else 'Success with MORE signatures',
        'Rejection with MOST signatures' if rejected_mean > law_active_mean else 'Rejection with LESS signatures', 
        'Rejected have consistently MORE' if diff_mean > 0 else 'Law Active have consistently MORE'
    ]
})

display(summary_comparison.style.hide(axis='index'))

print("\n💡 KEY TAKEAWAY FOR ORGANIZERS:")
key_takeaway = """
   Beyond 1 million signatures, collecting more doesn't significantly improve success rates.
   
   The Commission evaluates each ECI based on policy alignment, legal feasibility, 
   and current EU priorities. Since Commissioners are appointed rather than directly 
   elected, they respond to broader EU institutional dynamics rather than signature 
   counts alone. 
"""
print(key_takeaway)



📈 STATISTICAL SUMMARY
----------------------------------------------------------------------------------------------------


Outcome Category,Count,Min Signatures,Mean Signatures,Max Signatures,Key Insight
Law Active (Best),3,"1,054,973","1,261,794","1,659,543",Success with LEAST signatures
Rejected - Already Covered (Worst),3,"1,123,422","1,371,466","1,721,626",Rejection with MOST signatures
Difference,Same,"+68,449 more for rejected","+109,673 more for rejected","+62,083 more for rejected",Rejected have consistently MORE



💡 KEY TAKEAWAY FOR ORGANIZERS:

   Beyond 1 million signatures, collecting more doesn't significantly improve success rates.
   
   The Commission evaluates each ECI based on policy alignment, legal feasibility, 
   and current EU priorities. Since Commissioners are appointed rather than directly 
   elected, they respond to broader EU institutional dynamics rather than signature 
   counts alone. 



<a id='question-2'></a>
## <p style="padding:10px;background-color:#fff798;margin:0;color:#435672;font-family:newtimesroman;text-align:center;border-radius: 15px 50px;overflow:hidden;font-weight:500">2. What are the key temporal patterns from submission through implementation?</p>

**Visualizations:**

- Timeline waterfall: Collection → Verification → Response → Implementation
- Box plot: Response time distribution (successful vs rejected)
- Scatter: Verification duration vs outcome correlation
- Data tables: Median times by outcome, milestone durations

In [9]:
# Q2 Analysis Code
# Calculate timing metrics
df_initiatives['verification_days'] = (df_initiatives['timeline_verification_end'] - 
                                       df_initiatives['timeline_verification_start']).dt.days

df_initiatives['response_days'] = (df_initiatives['timeline_response_commission_date'] - 
                                   df_initiatives['timeline_collection_closed']).dt.days

# Response time by outcome (merger file)
if 'commission_submission_date' in df_merger.columns and 'law_implementation_date' in df_merger.columns:
    df_merger['commission_submission_date'] = pd.to_datetime(df_merger['commission_submission_date'], errors='coerce')
    df_merger['law_implementation_date'] = pd.to_datetime(df_merger['law_implementation_date'], errors='coerce')
    df_merger['response_lag_days'] = (df_merger['law_implementation_date'] - 
                                      df_merger['commission_submission_date']).dt.days
    
    print("Commission Response Lag Statistics:")
    print(df_merger[['registration_number', 'response_lag_days']].dropna().describe())

# Collection duration comparison
collection_by_success = df_initiatives.groupby('is_successful')['collection_days'].agg([
    ('count', 'count'),
    ('mean', 'mean'),
    ('median', 'median'),
    ('std', 'std')
]).round(1)
collection_by_success.index = ['Unsuccessful', 'Successful']

print("\nCollection Duration (Days) by Success:")
print(collection_by_success)

Commission Response Lag Statistics:
       response_lag_days
count           3.000000
mean         1808.000000
std          1473.196864
min           681.000000
25%           974.500000
50%          1268.000000
75%          2371.500000
max          3475.000000

Collection Duration (Days) by Success:
              count   mean  median    std
Unsuccessful     75  420.4   365.0  113.5
Successful       11  482.6   497.0  194.1


<a id='question-3'></a>
## <p style="padding:10px;background-color:#fff798;margin:0;color:#435672;font-family:newtimesroman;text-align:center;border-radius: 15px 50px;overflow:hidden;font-weight:500">3. How do optional Parliament actions correlate with Commission decisions?</p>

**Visualizations:**

- Histogram: Time from submission to Parliament hearing
- Histogram: Time from hearing to Commission decision
- Histogram: Time from plenary debate to Commission decision
- Stacked bar: Parliament engagement presence vs outcomes

In [10]:
# Q3 Analysis Code
# Parliament engagement from merger file
if 'parliament_hearing_date' in df_merger.columns:
    df_merger['parliament_hearing_date'] = pd.to_datetime(df_merger['parliament_hearing_date'], errors='coerce')
    df_merger['commission_submission_date'] = pd.to_datetime(df_merger['commission_submission_date'], errors='coerce')
    
    df_merger['hearing_to_submission_days'] = (df_merger['parliament_hearing_date'] - 
                                              df_merger['commission_submission_date']).dt.days
    
    print("Days from Submission to Parliament Hearing:")
    print(df_merger['hearing_to_submission_days'].describe())

# Plenary debate timing
if 'plenary_debate_date' in df_merger.columns:
    df_merger['plenary_debate_date'] = pd.to_datetime(df_merger['plenary_debate_date'], errors='coerce')
    df_merger['plenary_to_outcome_days'] = (df_merger['final_outcome_status'].notna().astype(int))
    
    # Check which ECIs have plenary debates
    has_plenary = df_merger['plenary_debate_date'].notna().sum()
    has_hearing = df_merger['parliament_hearing_date'].notna().sum()
    
    print(f"\nParliament Engagement:")
    print(f"  Hearings: {has_hearing}/{len(df_merger)} ECIs")
    print(f"  Plenary debates: {has_plenary}/{len(df_merger)} ECIs")
    
    # Compare outcomes by plenary presence
    plenary_by_outcome = df_merger.groupby(df_merger['plenary_debate_date'].notna())['final_outcome_status'].value_counts()
    print("\nOutcomes by Plenary Debate Presence:")
    print(plenary_by_outcome)

Days from Submission to Parliament Hearing:
count     11.000000
mean     111.181818
std       71.019460
min       41.000000
25%       64.000000
50%      109.000000
75%      119.000000
max      279.000000
Name: hearing_to_submission_days, dtype: float64

Parliament Engagement:
  Hearings: 11/11 ECIs
  Plenary debates: 7/11 ECIs

Outcomes by Plenary Debate Presence:
plenary_debate_date  final_outcome_status          
False                Law Active                        2
                     Rejected - Already Covered        1
                     Rejected - Alternative Actions    1
True                 Rejected - Already Covered        2
                     Action Plan Created               1
                     Being Studied                     1
                     Law Active                        1
                     Law Approved                      1
                     Law Promised                      1
Name: count, dtype: int64


<a id='question-4'></a>
## <p style="padding:10px;background-color:#fff798;margin:0;color:#435672;font-family:newtimesroman;text-align:center;border-radius: 15px 50px;overflow:hidden;font-weight:500">4. What funding patterns distinguish successful ECIs from unsuccessful ones?</p>

**Visualizations:**

- Grouped bar: Average funding (successful vs unsuccessful)
- Histogram: Number of sponsors distribution with success overlay
- Comparison bar: Private vs organizational sponsor success rates
- Stacked bar: Funding thresholds (€50k, €100k+) vs outcomes

In [11]:
# Q4 Analysis Code
# Convert funding_total to numeric first (FIX HERE)
if 'funding_total' in df_initiatives.columns:
    df_initiatives['funding_total'] = pd.to_numeric(
        df_initiatives['funding_total'], errors='coerce')

# Funding by success status
funding_by_success = df_initiatives.groupby('is_successful')['funding_total'].agg([
    ('count', 'count'),
    ('mean', 'mean'),
    ('median', 'median'),
    ('std', 'std'),
    ('max', 'max')
]).round(2)
funding_by_success.index = ['Unsuccessful', 'Successful']

print("Funding Statistics by Success:")
print(funding_by_success)

# Funding threshold analysis
df_initiatives['funding_level'] = pd.cut(df_initiatives['funding_total'],
    bins=[-1, 0, 50000, 100000, float('inf')],
    labels=['None', '€0-50k', '€50k-100k', '€100k+'])

funding_threshold = df_initiatives.groupby('funding_level')['is_successful'].agg([
    ('count', 'count'),
    ('successes', 'sum'),
    ('success_rate_%', lambda x: (x.sum()/len(x))*100)
]).round(1)

print("\nSuccess Rate by Funding Level:")
print(funding_threshold)

# Sponsor analysis if available
if 'funding_by' in df_initiatives.columns:
    # Parse funding sources (JSON)
    def count_sponsors(x):
        try:
            return len(json.loads(x)) if pd.notna(x) else 0
        except:
            return 0
    
    df_initiatives['sponsors_count'] = df_initiatives['funding_by'].apply(count_sponsors)
    
    sponsor_by_success = df_initiatives.groupby('is_successful')['sponsors_count'].agg([
        ('count', 'count'),
        ('mean', 'mean'),
        ('median', 'median')
    ]).round(1)
    sponsor_by_success.index = ['Unsuccessful', 'Successful']
    
    print("\nSponsor Count by Success:")
    print(sponsor_by_success)

Funding Statistics by Success:
              count   mean  median  std    max
Unsuccessful      1  500.0   500.0  NaN  500.0
Successful        0    NaN     NaN  NaN    NaN

Success Rate by Funding Level:
               count  successes  success_rate_%
funding_level                                  
None               0          0             NaN
€0-50k             1          0             0.0
€50k-100k          0          0             NaN
€100k+             0          0             NaN

Sponsor Count by Success:
              count  mean  median
Unsuccessful    110   2.0     0.0
Successful       11  21.9     8.0


<a id='question-5'></a>
## <p style="padding:10px;background-color:#fff798;margin:0;color:#435672;font-family:newtimesroman;text-align:center;border-radius: 15px 50px;overflow:hidden;font-weight:500">5. Which geographic strategies correlate with ECI success?</p>

**Visualizations:**

- Network/heatmap: Most frequent country combinations
- Comparison bar: Success rates when Germany/France meet thresholds
- Scatter: Number of countries vs outcome
- Box plot: Geographic diversity index by outcome

In [12]:
# Q5 Analysis Code
# Countries meeting threshold analysis
if 'signatures_collected_by_country' in df_initiatives.columns:
    def count_countries_met_threshold(sig_by_country_json):
        try:
            if pd.isna(sig_by_country_json):
                return 0
            data = json.loads(sig_by_country_json)
            countries_met = sum(1 for country, stats in data.items() 
                              if float(stats.get('percentage', 0)) >= 100)
            return countries_met
        except:
            return 0
    
    df_initiatives['countries_threshold_met'] = df_initiatives['signatures_collected_by_country'].apply(
        count_countries_met_threshold)
    
    # Convert to numeric to ensure proper aggregation
    df_initiatives['countries_threshold_met'] = pd.to_numeric(
        df_initiatives['countries_threshold_met'], errors='coerce').fillna(0).astype(int)
    
    # Countries by success
    country_by_success = df_initiatives.groupby('is_successful')['countries_threshold_met'].agg([
        ('count', 'count'),
        ('mean', 'mean'),
        ('median', 'median'),
        ('std', 'std')
    ]).round(1)
    country_by_success.index = ['Unsuccessful', 'Successful']
    
    print("Countries Meeting Threshold by Success:")
    print(country_by_success)
    
    # Success rate by country threshold distribution
    df_initiatives['country_category'] = pd.cut(
        df_initiatives['countries_threshold_met'],
        bins=[-0.1, 3, 7, 12, 30],
        labels=['0-3', '4-7', '8-12', '13+'],
        include_lowest=True)
    
    country_success = df_initiatives.groupby('country_category', observed=False, dropna=False)['is_successful'].agg([
        ('count', 'count'),
        ('successes', 'sum'),
        ('success_rate_%', lambda x: (x.sum()/len(x))*100 if len(x) > 0 else 0)
    ]).round(1)
    
    print("\nSuccess Rate by Country Diversity:")
    print(country_success)

Countries Meeting Threshold by Success:
              count  mean  median  std
Unsuccessful    110   0.0     0.0  0.0
Successful       11   0.0     0.0  0.0

Success Rate by Country Diversity:
                  count  successes  success_rate_%
country_category                                  
0-3                 121         11             9.1
4-7                   0          0             0.0
8-12                  0          0             0.0
13+                   0          0             0.0


<a id='question-6'></a>
## <p style="padding:10px;background-color:#fff798;margin:0;color:#435672;font-family:newtimesroman;text-align:center;border-radius: 15px 50px;overflow:hidden;font-weight:500">6. What organizational characteristics optimize ECI success?</p>

**Visualizations:**

- Histogram: Team size distribution with success overlay
- Bar: Multiple vs single representative success rates
- Comparison: Multi-country vs single-country teams
- Box plot: Optimal team size by outcome

In [13]:
# Q6 Analysis Code
# Parse organizer structure
def parse_organizer_count(org_json):
    try:
        if pd.isna(org_json):
            return 0
        data = json.loads(org_json)
        return data.get('number_of_people', 0)
    except:
        return 0

df_initiatives['organizer_count'] = df_initiatives['organizer_representative'].apply(parse_organizer_count)

# Convert to numeric and handle NaN
df_initiatives['organizer_count'] = pd.to_numeric(
    df_initiatives['organizer_count'], errors='coerce').fillna(0).astype(int)

# Team size by success
team_by_success = df_initiatives.groupby('is_successful')['organizer_count'].agg([
    ('count', 'count'),
    ('mean', 'mean'),
    ('median', 'median'),
    ('std', 'std')
]).round(1)
team_by_success.index = ['Unsuccessful', 'Successful']

print("Organizer Count by Success:")
print(team_by_success)

# Optimal team size
df_initiatives['team_category'] = pd.cut(df_initiatives['organizer_count'],
    bins=[-0.1, 2, 5, 10, float('inf')],
    labels=['1-2', '3-5', '6-10', '10+'],
    include_lowest=True)

team_success = df_initiatives.groupby('team_category', observed=False, dropna=False)['is_successful'].agg([
    ('count', 'count'),
    ('successes', 'sum'),
    ('success_rate_%', lambda x: (x.sum()/len(x))*100 if len(x) > 0 else 0)
]).round(1)

print("\nSuccess Rate by Team Size:")
print(team_success)

# Multi-country organizers
def has_international_team(org_json):
    try:
        if pd.isna(org_json):
            return False
        data = json.loads(org_json)
        countries = data.get('countries_of_residence', {})
        return len(countries) > 1
    except:
        return False

df_initiatives['is_international_team'] = df_initiatives['organizer_representative'].apply(has_international_team)

intl_success = df_initiatives.groupby('is_international_team')['is_successful'].agg([
    ('count', 'count'),
    ('successes', 'sum'),
    ('success_rate_%', lambda x: (x.sum()/len(x))*100 if len(x) > 0 else 0)
]).round(1)

# FIX: Only rename index if we have both groups
if len(intl_success) == 2:
    intl_success.index = ['Single Country', 'Multi-Country']
elif len(intl_success) == 1:
    # Check which group exists
    if intl_success.index[0] == False:
        intl_success.index = ['Single Country']
    else:
        intl_success.index = ['Multi-Country']

print("\nSuccess Rate by Team Internationality:")
print(intl_success)

Organizer Count by Success:
              count  mean  median  std
Unsuccessful    110   1.0     1.0  0.1
Successful       11   1.0     1.0  0.0

Success Rate by Team Size:
               count  successes  success_rate_%
team_category                                  
1-2              121         11             9.1
3-5                0          0             0.0
6-10               0          0             0.0
10+                0          0             0.0

Success Rate by Team Internationality:
                count  successes  success_rate_%
Single Country    121         11             9.1


<a id='question-7'></a>
## <p style="padding:10px;background-color:#fff798;margin:0;color:#435672;font-family:newtimesroman;text-align:center;border-radius: 15px 50px;overflow:hidden;font-weight:500">7. How do content features affect ECI outcomes?</p>

**Visualizations:**

- Bar: Annexes present vs absent success rates
- Box plot: Number of languages vs outcome
- Stacked bar: Existing legislation vs new frameworks
- Pie: Amendment vs new law requests

In [14]:
# Q7 Analysis Code
# Annexes analysis
df_initiatives['has_annex'] = df_initiatives['annex'].notna().astype(int)

annex_success = df_initiatives.groupby('has_annex')['is_successful'].agg([
    ('count', 'count'),
    ('successes', 'sum'),
    ('success_rate_%', lambda x: (x.sum()/len(x))*100)
]).round(1)
annex_success.index = ['No Annex', 'Has Annex']

print("Success Rate by Annex Presence:")
print(annex_success)

# Language availability
def count_languages(lang_str):
    try:
        if isinstance(lang_str, str) and lang_str.startswith('['):
            return len(eval(lang_str))
        elif isinstance(lang_str, str):
            return len(lang_str.split(','))
        return 0
    except:
        return 0

df_initiatives['language_count'] = df_initiatives['languages_available'].apply(count_languages)

lang_success = df_initiatives.groupby('is_successful')['language_count'].agg([
    ('count', 'count'),
    ('mean', 'mean'),
    ('median', 'median'),
    ('std', 'std')
]).round(1)
lang_success.index = ['Unsuccessful', 'Successful']

print("\nLanguage Count by Success:")
print(lang_success)

# Legislative target (existing vs new)
import re

def classify_legislation_target(objective, title):
    if pd.isna(objective):
        return 'Unknown'
    text = f"{title} {objective}".lower()
    
    # Check for explicit directive/regulation references
    if re.search(r'directive \d{4}/\d{1,3}', text):
        return 'Existing'
    if re.search(r'regulation \(eu\)', text):
        return 'Existing'
    if any(verb in text for verb in ['abrogate', 'amend', 'repeal']):
        return 'Existing'
    if any(verb in text for verb in ['propose legislation', 'establish', 'create']):
        return 'New'
    
    return 'Unclear'

df_initiatives['leg_target'] = df_initiatives.apply(
    lambda x: classify_legislation_target(x['objective'], x['title']), axis=1)

target_success = df_initiatives.groupby('leg_target')['is_successful'].agg([
    ('count', 'count'),
    ('successes', 'sum'),
    ('success_rate_%', lambda x: (x.sum()/len(x))*100)
]).round(1)

print("\nSuccess Rate by Legislative Target:")
print(target_success)

Success Rate by Annex Presence:
           count  successes  success_rate_%
No Annex      74          9            12.2
Has Annex     47          2             4.3

Language Count by Success:
              count  mean  median  std
Unsuccessful    110  24.0    24.0  0.0
Successful       11  24.0    24.0  0.0

Success Rate by Legislative Target:
            count  successes  success_rate_%
leg_target                                  
Existing       12          2            16.7
New            25          3            12.0
Unclear        84          6             7.1


<a id='question-8'></a>
## <p style="padding:10px;background-color:#fff798;margin:0;color:#435672;font-family:newtimesroman;text-align:center;border-radius: 15px 50px;overflow:hidden;font-weight:500">8. What Commission engagement patterns predict implementation success?</p>

**Visualizations:**

- Grouped bar: Commission official roles met vs outcomes
- Comparison bar: Deadline presence vs law implementation rate
- Multi-panel: Follow-up activity profile by outcome
- Stacked bar: Roadmaps/workshops vs implementation

In [15]:
# Q8 Analysis Code
# Commission engagement from merger file
if 'commission_officials_met' in df_merger.columns:
    has_officials = df_merger['commission_officials_met'].notna().sum()
    print(f"ECIs with Commission officials met: {has_officials}/{len(df_merger)}")

# Deadlines analysis
if 'commission_deadlines' in df_merger.columns:
    df_merger['has_deadline'] = df_merger['commission_deadlines'].notna().astype(int)
    
    deadline_by_outcome = df_merger.groupby('has_deadline')['final_outcome_status'].value_counts()
    print("\nOutcomes by Deadline Presence:")
    print(deadline_by_outcome)

# Follow-up actions
if 'has_followup_section' in df_merger.columns:
    followup_summary = pd.DataFrame({
        'Has Roadmap': [df_merger['has_roadmap'].sum()],
        'Has Workshop': [df_merger['has_workshop'].sum()],
        'Has Partnership': [df_merger['has_partnership_programs'].sum()],
        'Total Actions': [len(df_merger)]
    })
    
    print("\nFollow-up Actions Summary:")
    print(followup_summary)
    
    # Actions by outcome
    actions_by_outcome = df_merger.groupby('final_outcome_status').agg({
        'has_roadmap': 'sum',
        'has_workshop': 'sum',
        'has_partnership_programs': 'sum'
    })
    
    print("\nFollow-up Actions by Outcome:")
    print(actions_by_outcome)

ECIs with Commission officials met: 11/11

Outcomes by Deadline Presence:
has_deadline  final_outcome_status          
0             Rejected - Already Covered        3
              Law Active                        2
              Action Plan Created               1
              Rejected - Alternative Actions    1
1             Being Studied                     1
              Law Active                        1
              Law Approved                      1
              Law Promised                      1
Name: count, dtype: int64

Follow-up Actions Summary:
   Has Roadmap  Has Workshop  Has Partnership  Total Actions
0            1             3                4             11

Follow-up Actions by Outcome:
                                has_roadmap  has_workshop  \
final_outcome_status                                        
Action Plan Created                       1             1   
Being Studied                             0             0   
Law Active                    

<a id='question-9'></a>
## <p style="padding:10px;background-color:#fff798;margin:0;color:#435672;font-family:newtimesroman;text-align:center;border-radius: 15px 50px;overflow:hidden;font-weight:500">9. What commission response mechanisms characterize different outcomes?</p>

**Visualizations:**

- Box plot: Number of referenced legislation pieces by outcome
- Bar: Impact assessment presence vs outcomes
- Stacked bar: Stakeholder dialogue frequency
- Small multiples: Court cases presence and outcomes

In [16]:
# Q9 Analysis Code
# Referenced legislation
if 'referenced_legislation_by_name' in df_merger.columns:
    def count_referenced_legislation(ref_json):
        try:
            data = json.loads(ref_json)
            count = 0
            for category, items in data.items():
                if isinstance(items, list):
                    count += len(items)
                elif isinstance(items, dict):
                    count += len(items)
            return count
        except:
            return 0
    
    df_merger['referenced_leg_count'] = df_merger['referenced_legislation_by_name'].apply(count_referenced_legislation)
    
    leg_by_outcome = df_merger.groupby('final_outcome_status')['referenced_leg_count'].agg([
        ('count', 'count'),
        ('mean', 'mean'),
        ('median', 'median')
    ]).round(1)
    
    print("Referenced Legislation by Outcome:")
    print(leg_by_outcome)

# Impact assessments and stakeholder dialogue
if 'policies_actions' in df_merger.columns:
    def has_impact_assessment(actions_json):
        try:
            data = json.loads(actions_json)
            return any('impact assessment' in str(action).lower() for action in data)
        except:
            return False
    
    df_merger['has_assessment'] = df_merger['policies_actions'].apply(has_impact_assessment)
    
    assessment_by_outcome = df_merger.groupby('final_outcome_status')['has_assessment'].agg([
        ('count', 'count'),
        ('assessments', 'sum')
    ])
    
    print("\nImpact Assessments by Outcome:")
    print(assessment_by_outcome)

# Court cases
if 'court_cases_referenced' in df_merger.columns:
    has_court = df_merger['court_cases_referenced'].notna().sum()
    print(f"\nECIs with court cases referenced: {has_court}/{len(df_merger)}")

Referenced Legislation by Outcome:
                                count  mean  median
final_outcome_status                               
Action Plan Created                 1   3.0     3.0
Being Studied                       1   1.0     1.0
Law Active                          3   2.7     3.0
Law Approved                        1   2.0     2.0
Law Promised                        1   0.0     0.0
Rejected - Already Covered          3   2.0     2.0
Rejected - Alternative Actions      1   1.0     1.0

Impact Assessments by Outcome:
                                count  assessments
final_outcome_status                              
Action Plan Created                 1            0
Being Studied                       1            1
Law Active                          3            1
Law Approved                        1            1
Law Promised                        1            1
Rejected - Already Covered          3            0
Rejected - Alternative Actions      1            0

ECIs 

<a id='question-1'></a>
## <p style="padding:10px;background-color:#fff798;margin:0;color:#435672;font-family:newtimesroman;text-align:center;border-radius: 15px 50px;overflow:hidden;font-weight:500">10. What Are the Key Findings?</p>

**Summary Analysis (Descriptive Only):**

- Simple comparisons: means, medians, distributions
- Correlation matrices: relationships between key variables
- Visual profiles: radar charts, parallel coordinates
- Key factors: distinguish successful from unsuccessful ECIs
- ⚠️ NO PREDICTIONS: Purely exploratory analysis

In [17]:
# Q10 Analysis Code
# Overall success rate
overall_success_rate = (df_initiatives['is_successful'].sum() / len(df_initiatives)) * 100
print(f"Overall ECI Success Rate: {overall_success_rate:.1f}%")

# Comparative profile
profile_comparison = df_initiatives.groupby('is_successful').agg({
    'signatures_collected': ['mean', 'median'],
    'collection_days': ['mean', 'median'],
    'countries_threshold_met': 'mean',
    'organizer_count': 'mean',
    'language_count': 'mean',
    'funding_total': ['mean', 'median']
}).round(1)

profile_comparison.index = ['Unsuccessful', 'Successful']

print("\nSuccessful vs Unsuccessful ECI Profile Comparison:")
print(profile_comparison)

# Key distinguishing factors - FIX HERE
numeric_cols = df_initiatives.select_dtypes(include=[np.number]).columns.tolist()
# Filter columns with enough non-null data
numeric_cols = [col for col in numeric_cols if df_initiatives[col].notna().sum() > 10]

if numeric_cols and 'is_successful' in df_initiatives.columns:
    try:
        # Compute correlation matrix
        corr_data = df_initiatives[numeric_cols + ['is_successful']].copy()
        # Drop columns that are all NaN or have no variance
        corr_data = corr_data.loc[:, corr_data.notna().sum() > 10]
        
        correlation_matrix = corr_data.corr()
        
        if 'is_successful' in correlation_matrix.columns:
            correlation = correlation_matrix['is_successful'].drop('is_successful').sort_values(ascending=False)
            print("\nTop Factors Correlated with Success:")
            print(correlation.head(10))
    except Exception as e:
        print(f"\nCorrelation analysis skipped: {e}")

# Outcome distribution in successful ECIs
if 'df_merger' in locals() and len(df_merger) > 0:
    print("\nCommission Outcome Distribution (for successful ECIs):")
    print(df_merger['final_outcome_status'].value_counts())
    
    # Implementation rate
    impl_rate = df_merger['law_implementation_date'].notna().sum() / len(df_merger) * 100
    print(f"\nLaw Implementation Rate: {impl_rate:.1f}%")

Overall ECI Success Rate: 9.1%


TypeError: agg function failed [how->mean,dtype->object]

<a id='question-1'></a>
## <p style="padding:10px;background-color:#fff798;margin:0;color:#435672;font-family:newtimesroman;text-align:center;border-radius: 15px 50px;overflow:hidden;font-weight:500">💾 Export Analysis Results to CSV</p>

In [ ]:
# Export all Q1-Q10 results
def export_all_results(df_initiatives, df_merger, output_dir='eda_data_output'):
    import os
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    # Q1 Results
    df_initiatives['final_outcome'].value_counts().to_csv(f'{output_dir}/Q1_outcome_distribution.csv')
    
    # Q2 Temporal
    temp_data = df_initiatives.groupby('is_successful')['collection_days'].agg([
        ('Count', 'count'),
        ('Mean_Days', 'mean'),
        ('Median_Days', 'median')
    ])
    temp_data.index = ['Unsuccessful', 'Successful']
    temp_data.to_csv(f'{output_dir}/Q2_temporal_patterns.csv')
    
    # Q3 Parliament (from merger)
    if 'parliament_hearing_date' in df_merger.columns:
        parliament_data = df_merger.groupby(df_merger['parliament_hearing_date'].notna())['final_outcome_status'].value_counts()
        parliament_data.to_csv(f'{output_dir}/Q3_parliament_engagement.csv')
    
    # Q4 Funding
    funding_data = df_initiatives.groupby('is_successful')['funding_total'].agg([
        ('Count', 'count'),
        ('Mean_EUR', 'mean'),
        ('Median_EUR', 'median'),
        ('Max_EUR', 'max')
    ])
    funding_data.index = ['Unsuccessful', 'Successful']
    funding_data.to_csv(f'{output_dir}/Q4_funding_patterns.csv')
    
    # Q5 Geographic
    if 'countries_threshold_met' in df_initiatives.columns:
        geo_data = df_initiatives.groupby('is_successful')['countries_threshold_met'].agg([
            ('Count', 'count'),
            ('Mean_Countries', 'mean'),
            ('Median_Countries', 'median')
        ])
        geo_data.index = ['Unsuccessful', 'Successful']
        geo_data.to_csv(f'{output_dir}/Q5_geographic_patterns.csv')
    
    # Q6 Organizational
    if 'organizer_count' in df_initiatives.columns:
        org_data = df_initiatives.groupby('is_successful')['organizer_count'].agg([
            ('Count', 'count'),
            ('Mean_Organizers', 'mean'),
            ('Median_Organizers', 'median')
        ])
        org_data.index = ['Unsuccessful', 'Successful']
        org_data.to_csv(f'{output_dir}/Q6_organizational_characteristics.csv')
    
    # Q7 Content
    if 'leg_target' in df_initiatives.columns:
        content_data = df_initiatives.groupby('leg_target')['is_successful'].agg([
            ('Count', 'count'),
            ('Successes', 'sum'),
            ('Success_Rate_%', lambda x: (x.sum()/len(x))*100)
        ]).round(1)
        content_data.to_csv(f'{output_dir}/Q7_content_features.csv')
    
    # Q8 Commission Engagement (from merger)
    if 'has_deadline' in df_merger.columns:
        commission_data = df_merger.groupby('has_deadline')['final_outcome_status'].value_counts()
        commission_data.to_csv(f'{output_dir}/Q8_commission_engagement.csv')
    
    # Q9 Response Mechanisms
    if 'referenced_leg_count' in df_merger.columns:
        response_data = df_merger.groupby('final_outcome_status')['referenced_leg_count'].agg([
            ('Count', 'count'),
            ('Mean_References', 'mean')
        ]).round(1)
        response_data.to_csv(f'{output_dir}/Q9_response_mechanisms.csv')
    
    # Q10 Summary
    summary_data = df_initiatives.groupby('is_successful').agg({
        'signatures_collected': 'mean',
        'collection_days': 'mean',
        'funding_total': 'mean'
    }).round(2)
    summary_data.index = ['Unsuccessful', 'Successful']
    summary_data.to_csv(f'{output_dir}/Q10_key_findings_summary.csv')
    
    print(f"✓ All results exported to '{output_dir}/' directory")
    print(f"✓ Files created: Q1_outcome_distribution.csv through Q10_key_findings_summary.csv")

# Run export function
export_all_results(df_initiatives, df_merger)